# Load data

In [ ]:
##Some Code to import your corpus

# Import

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Preprocess

## data Lst20

In [ ]:
path = '/content/'
wordss = []
poss = []
counter = 0
word = []
pos = []
for dir in sorted(os.listdir(path)):
    if dir != ".DS_Store" and dir.startswith('T'):
      f = open(os.path.join(path,dir), "r")
      for i in f :
        if len(i.split()) == 0 :
          wordss.append(word)
          poss.append(pos)
          word = []
          pos = []
          continue
        word.append(i.split("\t")[0])
        pos.append(i.split("\t")[2])

In [ ]:
wordss = list(filter(lambda x: len(x) <= 150, wordss))
poss=list(filter(lambda x: len(x) <= 150, poss))

Split train/test data and preprocessing data

In [ ]:

(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(wordss, poss, test_size=0.2)

tags=['B_BRN', 'B_DES', 'B_DTM', 'B_LOC', 'B_MEA', 'B_NUM', 'B_ORG', 'B_PER', 
      'B_TRM', 'B_TTL', 'E_BRN', 'E_DES', 'E_DTM', 'E_LOC', 'E_MEA', 'E_NUM', 
      'E_ORG', 'E_PER', 'E_TRM', 'E_TTL', 'I_BRN', 'I_DES', 'I_DTM', 'I_LOC', 
      'I_MEA', 'I_NUM', 'I_ORG', 'I_PER', 'I_TRM', 'I_TTL', 'O']

for ts in range(len(poss)):
  if poss[ts]=='ORG_I':
    poss[ts]='I_ORG'

  elif poss[ts]=='LOC_I':
    poss[ts]='I_LOC'

  elif poss[ts]=='PER_I':
    poss[ts]='I_PER'

  elif poss[ts] not in tags:
    poss[ts]='O'
for s in range(len(wordss)) :
    for w in range(len(wordss[s])):
        try:
          a=float(wordss[s][w]) +1
          wordss[s][w]='NUM'
        except:
          pass

#Word2index

In [ ]:
##MakeDictionary
word2index={}
word2index_save=[]
tag2index = {t: i + 1 for i, t in enumerate(tags)}
tag2index['PAD'] = 0  

for i, sent in enumerate(wordss):
  for j,word in enumerate(sent):
    word2index[word]=j
# word2index = { w :i + 2 for i, w in enumerate(wordss)}
word2index['PAD'] = 0  
word2index['UNK'] = 1   

word2index_save = list((w ,i + 2 ) for i, w in enumerate(wordss))
word2index_save.append(('PAD',0))
word2index_save.append(('UNK',1))

In [ ]:
df=pd.DataFrame(word2index_save,columns=['words','indexs'])
df.to_csv('my_dict.csv')
df

,words,indexs
0,"[สุรยุทธ์, ยัน, ปฏิเสธ, ลงนาม, _, MOU, _, กับ,...",2
1,"[NUM, _, กันยายน, _, NUM, _, 12:21, _, น.]",3
2,"[นายก, ฯ, _, ย้ำ, ไม่, ลงนาม, ใน, _, MOU, _, ร...",4
3,"[ยัน, ไทย, ไม่, ได้, ล้มเหลว, ถึง, ขั้น, นั้น,...",5
4,"[พร้อม, หนุน, กกต., จัด, เลือกตั้ง, ให้, บริสุ...",6
...,...,...
73747,"[**, _, เชิญ, "", ชวรัตน์, "", _, ชี้แจง, _, NUM...",73749
73748,"[นาย, ไพโรจน์, _, กล่าว, ด้วย, ว่า, _, คณะ, กร...",73750
73749,"[ด้าน, นาย, ฐนโรจน์, _, กล่าว, ว่า, _, ตน, ใน,...",73751
73750,PAD,0


In [ ]:
b=['B_BRN', 'B_DES', 'B_DTM', 'B_LOC', 'B_MEA', 'B_NUM', 'B_ORG', 'B_PER', 'B_TRM', 'B_TTL', 'E_BRN', 'E_DES', 'E_DTM', 'E_LOC', 'E_MEA', 'E_NUM', 'E_ORG', 'E_PER', 'E_TRM', 'E_TTL', 'I_BRN', 'I_DES', 'I_DTM', 'I_LOC', 'I_MEA', 'I_NUM', 'I_ORG', 'I_PER', 'I_TRM', 'I_TTL', 'O']
c=[]
for i in tag2index:
  if i not in b:
    c.append(i)
print(c)

['PAD']


In [ ]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w]) #.lower()])
        except KeyError:
            s_int.append(word2index['UNK'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w]) #.lower()])
        except KeyError:
            s_int.append(word2index['UNK'])
 
    test_sentences_X.append(s_int)

for s in train_tags:
    a=[]
    for t in s:
        try:
            a.append(tag2index[t])
        except:
            a.append(tag2index['UNK'])
    train_tags_y.append(a)
for s in test_tags:
    #print(s)
    a=[]
    for t in s:
        try:
            a.append(tag2index[t])
        except:
            a.append(tag2index['UNK'])
    test_tags_y.append(a)

# Model

In [ ]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)  # 271

150


In [ ]:
##Padding if there are less than 150 word
from keras.preprocessing.sequence import pad_sequences
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 128)          6871552   
                                                                 
 bidirectional (Bidirectiona  (None, 150, 512)         788480    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 150, 43)          22059     
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 150, 43)           0         
                                                                 
Total params: 7,682,091
Trainable params: 7,682,091
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [ ]:
train_tags_y, train_tags_y.shape

(array([[42, 42, 42, ...,  0,  0,  0],
        [42, 42, 42, ...,  0,  0,  0],
        [42, 42, 42, ...,  0,  0,  0],
        ...,
        [42, 42, 42, ...,  0,  0,  0],
        [42, 42, 42, ...,  0,  0,  0],
        [42, 42, 42, ...,  0,  0,  0]], dtype=int32), (59000, 150))

In [ ]:
train_tags_y[0]
print(len(tag2index))

43


In [ ]:

cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=32, epochs=5, validation_split=0.2)

Epoch 1/5
1475/1475 [==============================] - 165s 107ms/step - loss: 0.1200 - accuracy: 0.9711 - val_loss: 0.0456 - val_accuracy: 0.9851
Epoch 2/5
1475/1475 [==============================] - 155s 105ms/step - loss: 0.0360 - accuracy: 0.9880 - val_loss: 0.0383 - val_accuracy: 0.9873
Epoch 3/5
1475/1475 [==============================] - 156s 106ms/step - loss: 0.0275 - accuracy: 0.9905 - val_loss: 0.0363 - val_accuracy: 0.9878
Epoch 4/5
1475/1475 [==============================] - 156s 106ms/step - loss: 0.0231 - accuracy: 0.9919 - val_loss: 0.0377 - val_accuracy: 0.9877
Epoch 5/5
1475/1475 [==============================] - 155s 105ms/step - loss: 0.0200 - accuracy: 0.9929 - val_loss: 0.0372 - val_accuracy: 0.9883


In [ ]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

461/461 [==============================] - 14s 31ms/step - loss: 0.0368 - accuracy: 0.9884
accuracy: 98.84171485900879


In [ ]:
model.save('model_NER.h5')

# Predict

In [ ]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
            
        token_sequences.append(token_sequence)
 
    return token_sequences

## try

In [ ]:
a='และ ยืนยัน ว่า ประชาชน ใน ภาค ใต้ ไม่ มีการ ซื้อ เสียง'
test_samples=[a.split()]
print(test_samples)# = [['นาง','สุดารัตน์','เกยุราพันธ์','รมว.','สาธารณสุข']]

[['และ', 'ยืนยัน', 'ว่า', 'ประชาชน', 'ใน', 'ภาค', 'ใต้', 'ไม่', 'มีการ', 'ซื้อ', 'เสียง']]


In [ ]:
test_samples_X = []
for s in test_samples:
    print(s)
    s_int = []
    
    for w in s:
        try:
            s_int.append(word2index[w]) #.lower()])
            print(s_int)
        except KeyError:
            s_int.append(word2index['-UNK-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

['และ', 'ยืนยัน', 'ว่า', 'ประชาชน', 'ใน', 'ภาค', 'ใต้', 'ไม่', 'มีการ', 'ซื้อ', 'เสียง']
[31975]
[31975, 3412]
[31975, 3412, 2925]
[31975, 3412, 2925, 51059]
[31975, 3412, 2925, 51059, 14066]
[31975, 3412, 2925, 51059, 14066, 2814]
[31975, 3412, 2925, 51059, 14066, 2814, 5209]
[31975, 3412, 2925, 51059, 14066, 2814, 5209, 34052]
[31975, 3412, 2925, 51059, 14066, 2814, 5209, 34052, 41474]
[31975, 3412, 2925, 51059, 14066, 2814, 5209, 34052, 41474, 6684]
[31975, 3412, 2925, 51059, 14066, 2814, 5209, 34052, 41474, 6684, 51292]
[[31975  3412  2925 51059 14066  2814  5209 34052 41474  6684 51292     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0 

In [ ]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[2.1642062e-07 1.1774106e-07 6.9564152e-08 ... 1.9272306e-05
   3.4463611e-07 9.9989855e-01]
  [3.6144130e-09 1.3884236e-09 9.9120612e-10 ... 1.0755753e-08
   5.4350329e-09 9.9999905e-01]
  [2.1949864e-08 1.2659463e-08 8.5253182e-09 ... 3.6690949e-06
   5.4401195e-08 9.9998224e-01]
  ...
  [1.0000000e+00 4.5987898e-11 9.8159224e-12 ... 1.7389731e-12
   1.4270700e-11 1.4406609e-09]
  [1.0000000e+00 4.0892054e-10 1.1451667e-10 ... 3.6003130e-12
   1.2117168e-10 4.8142373e-10]
  [9.9999678e-01 1.2837711e-08 4.4021529e-09 ... 6.2132355e-11
   4.4859214e-09 1.7688734e-09]]] (1, 150, 43)


In [ ]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'

## Submission1

In [ ]:
path_test = '/content/ne_test.txt'
ne_test_pd = pd.read_csv(path_test, sep=' /', header=None, skip_blank_lines=False)
ne_test_pd=ne_test_pd.fillna('_')

In [ ]:
test_samples_X = []

check_predict=[]
for s in ne_test_pd[0]:
    try:
        test_samples_X.append(word2index[s]) #.lower()])
        check_predict.append(s)
    except KeyError:
        test_samples_X.append(word2index['-UNK-'])
        check_predict.append('-UNK-')

nu_test_sent = []
check_pred=[]
for i in range(0,len(test_samples_X),MAX_LENGTH):
  nu_test_sent.append(test_samples_X[i:i+MAX_LENGTH])
  check_pred.append(check_predict[i:i+MAX_LENGTH])

a=len(nu_test_sent[-1]) 
b=len(nu_test_sent)
for i in range(MAX_LENGTH-a):
  nu_test_sent[-1].append(word2index['-PAD-']) # PAD last index
  check_pred[-1].append('-PAD-')

In [ ]:
nu_test_sent=np.array(nu_test_sent)
predictions = model.predict(nu_test_sent)
print(predictions, predictions.shape)

In [ ]:
submis=logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

In [ ]:
sample_submission=[]
for i in submis:
  for j in i:
    # if j=='-PAD-': break
    sample_submission.append(j)


In [ ]:
len(sample_submission)

69600

In [ ]:
len(test_samples_X)

69562

In [ ]:
sub  =  sample_submission[0:len(test_samples_X)-1]
print(len(sub))

69561


In [ ]:
num_csv=np.arange(69562).tolist()
print(num_csv[-1])

In [ ]:
ne_test_pd

,0
0,นาง
1,สุดารัตน์
2,_
3,เกยุราพันธ์
4,_
...,...
69557,เหตุ
69558,ดัง
69559,กล่าว
69560,ขึ้น


In [ ]:
sub_csv = []
for n, i in enumerate(sub):
  sub_csv.append([n+1, i])

In [ ]:
pd.DataFrame(sub_csv, columns=['Id','Predicted']).to_csv(f'submission.csv', index=None)

In [ ]:
df=pd.DataFrame([num_csv,sub],columns=['Id','Predicted'],index=False)
df.to_csv('sample_submission.csv')

In [ ]:
print(check_pred[0])
print(submis[0])

['นาง', 'สุดารัตน์', '_', 'เกยุราพันธ์', '_', 'รมว.', 'สาธารณสุข', '_', 'แถลง', 'ถึง', 'การ', 'จัด', 'งาน', 'มหกรรม', 'กิน', 'ไก่', 'ไทย', 'ปลอด', 'ภัย', '_', '100', '%', '_', 'ว่า', '_', 'รัฐบาล', 'ร่วม', 'กับ', 'กทม.', 'จัด', 'งาน', 'ดัง', 'กล่าว', 'ขึ้น', 'ที่', 'ท้องสนามหลวง', 'ใน', 'วัน', 'เสาร์', 'ที่', '_', '7', '_', 'ก.พ.', 'นี้', '_', 'ตั้งแต่', '_', '16.00', 'น.', 'เป็นต้นไป', '_', 'ส่วน', 'พ.ต.ท.', 'ทักษิณ', '_', 'ชินวัตร', 'รัฐมนตรี', '_', 'จะ', 'เดินทาง', 'มา', 'ถึง', 'ที่', 'งาน', 'ใน', 'เวลา', '_', '16.30', 'น.', '_', 'เพื่อ', 'เยี่ยมชม', 'นิทรรศการ', '_', 'และ', 'ประกอบ', 'อาหาร', 'เมนู', '_', '-UNK-', 'ไก่', '-UNK-', 'สุก', '_', 'ไก่', 'ทอด', '_', 'ด้วย', 'ตัว', 'เอง', 'และ', 'แจกจ่าย', 'ให้', 'กับ', 'ประชาชน', '_', 'โดย', 'จะ', 'มี', 'นาย', 'สมัคร', '_', 'สุนทรเวช', '_', 'ผู้', 'ว่า', 'ฯ', 'กทม.', '_', 'และ', 'ม.ล.', '-UNK-', '_', 'สวัสดิวัฒน์', '_', 'หรือ', '-UNK-', '_', 'ร่วม', 'ปรุง', 'อาหาร', 'ด้วย', '_', 'ภาย', 'หลัง', 'ที่', 'นายก', 'ฯ', 'ปรุง', 'อาหาร', 'เสร็จ'